### Learning How to Use Azure ML

In [1]:
# connect to a workspace
from azureml.core import Workspace

ws = Workspace.from_config()

In [2]:
# retrieve a dictionary object containing the compute targets defined in the workspace
for compute_name in ws.compute_targets:
    compute = ws.compute_targets[compute_name]
    print(compute.name, ":", compute.type)

NLP-Learn-com4 : AmlCompute
cpucluster : AmlCompute


In [3]:
## Experiment run context
from azureml.core import Experiment

# create an experiment variable
experiment = Experiment(workspace = ws, name = "my-experiment")

# start the experiment
run = experiment.start_logging()

# experiment code goes here

# end the experiment
run.complete()

In [ ]:
## Logging Metrics and Creating Output

# logging metrics
from azureml.core import Experiment
import pandas as pd

# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace = ws, name = 'my-experiment')

# Start logging data from the experiment
run = experiment.start_logging()

# load the dataset and count the rows
data = pd.read_csv('data.csv')
row_count = (len(data))

# Log the row count
run.log('observations', row_count)

# Complete the experiment
run.complete()

# retrieving and viewing logged metrics
from azureml.widgets import RunDetails

RunDetails(run).show()

# can also use json and get json output
import json

# Get logged metrics
metrics = run.get_metrics()
print(json.dumps(metrics, indent=2))

In [ ]:
# experiment output files
run.upload_file(name='outputs/sample.csv', path_or_stream='./sample.csv')

When running an experiment in a remote compute context, any files written to the outputs folder in the compute context are automatically uploaded to the run's outputs folder when the run completes.

In [ ]:
# retrieve list of output files
import json

files = run.get_file_names()
print(json.dumps(files, indent=2))

In [ ]:
## Running the script as an experiment
from azureml.core import Run
import pandas as pd
import matplotlib.pyplot as plt
import os

# Get the experiment run context
run = Run.get_context()

# load the diabetes dataset
data = pd.read_csv('data.csv')

# Count the rows and log the result
row_count = (len(data))
run.log('observations', row_count)

# Save a sample of the data
os.makedirs('outputs', exist_ok=True)
data.sample(100).to_csv("outputs/sample.csv", index=False, header=True)

# Complete the run
run.complete()

In [ ]:
from azureml.core import Experiment, RunConfiguration, ScriptRunConfig

# create a new RunConfig object
experiment_run_config = RunConfiguration()

# Create a script config
script_config = ScriptRunConfig(source_directory=experiment_folder, 
                      script='experiment.py',
                      run_config=experiment_run_config) 

# submit the experiment
experiment = Experiment(workspace = ws, name = 'my-experiment')
run = experiment.submit(config=script_config)
run.wait_for_completion(show_output=True)